<a href="https://colab.research.google.com/github/masdesouza/FDIA-PdM/blob/master/notebooks/regression_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import modules

#2. Configuration

##2.1 Import dataset

# 3. Data Ingestion

#4. Data Preprocessing

## 4.1 Train

## 4.2 Test


#5. Modeling

#6. EVALUATE ON TEST DATA
